# Imports

In [13]:
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, LongType, StringType

In [2]:
spark = (SparkSession
         .builder
         .appName("Basic Structured Operations")
         .getOrCreate())

In [3]:
spark

# Data

In [5]:
schema = StructType([
  StructField("DEST_COUNTRY_NAME", StringType(), True),
  StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
  StructField("count", LongType(), False, metadata={"hello":"world"})
])
df = (spark
      .read
      .format("json")
      .schema(schema)
      .load("data/flight-data/json/2015-summary.json"))
df.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

# Common Operations

In [6]:
df.count()

256

In [8]:
F.col("count")

Column<b'count'>

In [9]:
F.column("count")

Column<b'count'>

In [10]:
F.expr("count")

Column<b'count'>

In [ ]:
expr("(((someCol + 5) * 200) - 6) < otherCol")

In [15]:
myRow = Row("Hello", None, 1, False)

In [16]:
myRow[0]

'Hello'

In [17]:
myRow[2]

1

In [18]:
df.createOrReplaceTempView("df_table")

In [19]:
myManualSchema = StructType([
  StructField("some", StringType(), True),
  StructField("col", StringType(), True),
  StructField("names", LongType(), False)
])
myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



In [20]:
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [21]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [23]:
df.select(
    F.expr("DEST_COUNTRY_NAME"),
    F.col("DEST_COUNTRY_NAME"),
    F.column("DEST_COUNTRY_NAME"))\
  .show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [25]:
df.select(F.expr("DEST_COUNTRY_NAME AS destination")).show(2)

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [27]:
(df.select(F.expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME"))
 .show(2))

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [28]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [29]:
df.selectExpr(
  "*", # all original columns
  "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
  .show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [30]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)

+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



In [33]:
df.select(F.expr("*"), F.lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



In [35]:
df.withColumn("numberOne", F.lit(1)).show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [38]:
(df.withColumn("withinCountry", F.expr("ORIGIN_COUNTRY_NAME = DEST_COUNTRY_NAME"))
 .show(2))

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [40]:
(df.selectExpr("*", "ORIGIN_COUNTRY_NAME = DEST_COUNTRY_NAME AS withinCountry")
 .show(2))

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [39]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns

['dest', 'ORIGIN_COUNTRY_NAME', 'count']

In [41]:
# If we have spaces in column name, we can use `column name`
# dfWithLongColName.selectExpr(
#     "`This Long Column-Name`",
#     "`This Long Column-Name` as `new col`")\
#   .show(2)

# dfWithLongColName.select(expr("`This Long Column-Name`")).columns

In [45]:
(df
 .where((F.col("count") < 2) & (F.col("ORIGIN_COUNTRY_NAME") != "Croatia"))
 .explain())

== Physical Plan ==
*(1) Project [DEST_COUNTRY_NAME#0, ORIGIN_COUNTRY_NAME#1, count#2L]
+- *(1) Filter (((isnotnull(count#2L) AND isnotnull(ORIGIN_COUNTRY_NAME#1)) AND (count#2L < 2)) AND NOT (ORIGIN_COUNTRY_NAME#1 = Croatia))
   +- FileScan json [DEST_COUNTRY_NAME#0,ORIGIN_COUNTRY_NAME#1,count#2L] Batched: false, DataFilters: [isnotnull(count#2L), isnotnull(ORIGIN_COUNTRY_NAME#1), (count#2L < 2), NOT (ORIGIN_COUNTRY_NAME#..., Format: JSON, Location: InMemoryFileIndex[file:/Users/imad/Documents/courses/data-engineering/big-data/notebooks/Spark-De..., PartitionFilters: [], PushedFilters: [IsNotNull(count), IsNotNull(ORIGIN_COUNTRY_NAME), LessThan(count,2), Not(EqualTo(ORIGIN_COUNTRY_..., ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>




In [44]:
(df
 .where(F.col("count") < 2)
 .where(F.col("ORIGIN_COUNTRY_NAME") != "Croatia")
 .explain())

== Physical Plan ==
*(1) Project [DEST_COUNTRY_NAME#0, ORIGIN_COUNTRY_NAME#1, count#2L]
+- *(1) Filter (((isnotnull(count#2L) AND isnotnull(ORIGIN_COUNTRY_NAME#1)) AND (count#2L < 2)) AND NOT (ORIGIN_COUNTRY_NAME#1 = Croatia))
   +- FileScan json [DEST_COUNTRY_NAME#0,ORIGIN_COUNTRY_NAME#1,count#2L] Batched: false, DataFilters: [isnotnull(count#2L), isnotnull(ORIGIN_COUNTRY_NAME#1), (count#2L < 2), NOT (ORIGIN_COUNTRY_NAME#..., Format: JSON, Location: InMemoryFileIndex[file:/Users/imad/Documents/courses/data-engineering/big-data/notebooks/Spark-De..., PartitionFilters: [], PushedFilters: [IsNotNull(count), IsNotNull(ORIGIN_COUNTRY_NAME), LessThan(count,2), Not(EqualTo(ORIGIN_COUNTRY_..., ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>




In [54]:
(df
 .where(F.expr("count < 2"))
 .where(F.expr("ORIGIN_COUNTRY_NAME != 'Croatia'"))
 .explain())

== Physical Plan ==
*(1) Project [DEST_COUNTRY_NAME#0, ORIGIN_COUNTRY_NAME#1, count#2L]
+- *(1) Filter (((isnotnull(count#2L) AND isnotnull(ORIGIN_COUNTRY_NAME#1)) AND (count#2L < 2)) AND NOT (ORIGIN_COUNTRY_NAME#1 = Croatia))
   +- FileScan json [DEST_COUNTRY_NAME#0,ORIGIN_COUNTRY_NAME#1,count#2L] Batched: false, DataFilters: [isnotnull(count#2L), isnotnull(ORIGIN_COUNTRY_NAME#1), (count#2L < 2), NOT (ORIGIN_COUNTRY_NAME#..., Format: JSON, Location: InMemoryFileIndex[file:/Users/imad/Documents/courses/data-engineering/big-data/notebooks/Spark-De..., PartitionFilters: [], PushedFilters: [IsNotNull(count), IsNotNull(ORIGIN_COUNTRY_NAME), LessThan(count,2), Not(EqualTo(ORIGIN_COUNTRY_..., ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>




In [59]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

256

In [60]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()

125

In [61]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

138

In [62]:
dataFrames = df.randomSplit([0.25, 0.75], seed)
dataFrames[0].count() > dataFrames[1].count() # False

False

In [67]:
schema = df.schema
newRows = [
  Row("New Country", "Other Country", 5),
  Row("New Country 2", "Other Country 3", 1)
]
parallelizedRows = spark.sparkContext.parallelize(newRows)
newDF = spark.createDataFrame(parallelizedRows, schema)

df.union(newDF)\
  .where("count = 1")\
  .where(F.col("ORIGIN_COUNTRY_NAME") != "United States")\
  .show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



In [ ]:
df.sort("count").show(5)
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

df.orderBy(expr("count desc")).show(2)
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)

spark.read.format("json").load("/data/flight-data/json/*-summary.json")\
  .sortWithinPartitions("count")

df.limit(5).show()

df.orderBy(expr("count desc")).limit(6).show()

In [ ]:
df.rdd.getNumPartitions() # 1

df.repartition(5)

df.repartition(col("DEST_COUNTRY_NAME"))

df.repartition(5, col("DEST_COUNTRY_NAME"))

df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

In [68]:
collectDF = df.limit(10)

In [69]:
collectDF.take(5) # take works with an Integer count

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62)]

In [70]:
collectDF.show() # this prints it out nicely

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



In [71]:
collectDF.show(5, False)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



In [72]:
collectDF.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [73]:
spark.stop()